In [1]:
import watertap as wt
import pandas as pd
from pyomo.environ import value, Block
from idaes.core import FlowsheetBlock

from case_study_trains import *
import case_study_trains

m = wt.watertap_setup(dynamic = False)

# wt.case_study_trains.train = {"case_study": "KBHDP",
#                              "reference": "NAWI",
#                              "scenario": "Baseline"}

wt.case_study_trains.train = {"case_study": "Irwin",
                             "reference": "NAWI",
                             "scenario": "Baseline"}

# wt.case_study_trains.train = {"case_study": "EMWD",
#                              "reference": "NAWI",
#                              "scenario": "Baseline"}

# wt.case_study_trains.train = {"case_study": "Tampa_Bay",
#                              "reference": "NAWI",
#                              "scenario": "Baseline"}


# TODO LATER: how to make this sync with info in train input data. We might not need to do that.
#But, if the source water type is different to what is in the train (pfd dictionary), 
#then we should updat the node name. If more than two sources - what to do? Needs to be
#based on pfd node!?

# wt.case_study_trains.source_water = {"case_study": "KBHDP", 
#                              "reference": "NAWI",
#                              "scenario": "Baseline",
#                              "water_type": "KBHDP_Brackish_Ave"}

wt.case_study_trains.source_water = {"case_study": "Irwin", 
                             "reference": "NAWI",
                             "scenario": "Baseline",
                             "water_type": "Irwin_Brackish"}

# wt.case_study_trains.source_water = {"case_study": "EMWD", 
#                              "reference": "NAWI",
#                              "scenario": "Baseline",
#                              "water_type": "EMWD_CA_Brackish"}

# wt.case_study_trains.source_water = {"case_study": "Tampa_Bay", 
#                              "reference": "NAWI",
#                              "scenario": "Baseline",
#                              "water_type": "Seawater"}

m = wt.case_study_trains.get_case_study(m = m) # flow is set as case study flow unless defined.

Irwin
well_field
treated_storage
media_filtration
hydrochloric_acid_addition
anti_scalant_addition
sodium_bisulfite_addition
electrodialysis_reversal
water_pumping_station
treated_storage_b
lime_softening
media_filtration_2
ro_deep
irwin_brine_management
municipal_drinking
deep_well_injection
surface_discharge


In [2]:
wt.display.show_train2(model_name=m)

In [4]:
wt.run_water_tap(m = m, solver_results = True, print_model_results = True)

degrees_of_freedom: 0
Ipopt 3.12.12: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.12, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     2648
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      641

Total number of variables............................:      833
                     variables with only lower bounds:       32
                variables with lower and upper bounds:        1
                     variables with only uppe

In [4]:
# m.fs.irwin_brine_management.get_costing()

In [5]:
# value(m.fs.treated_storage.inlet.flow_vol[0.0])

In [5]:
train = wt.case_study_trains.train

py_var = [
    "fixed_cap_inv",
    "total_cap_investment",
    "cat_and_chem_cost",
    "electricity_cost",
    "total_fixed_op_cost"]

excel_var = ['Fixed Capital Investment (FCI)',
            'Total Capital Investment (TCI)',
            'Catalysts and Chemicals',
            'Electricity',
            'Total Fixed Operating Costs']



big_dict = {}
# for letter in ['A', 'B', 'C']:
for b_unit in m.fs.component_objects(Block, descend_into=True):
    unit = str(b_unit)[3:]
    if hasattr(b_unit, 'costing'):
        print(f'____________{unit}____________')
        py_vals =  []
        excel_vals = []
        big_dict[unit] = {'python': {}, 'excel': {}}
#         print(f'\nflow in --> {round(value(b_unit.flow_vol_in[0]), 4)}')
#         print(f'flow out --> {round(value(b_unit.flow_vol_out[0]), 4)}\n')
        print(f'\nflow in --> {round(value(b_unit.inlet.flow_vol[0.0]), 4) * 3600}')
        print(f'flow out --> {round(value(b_unit.outlet.flow_vol[0.0]), 4) * 3600}')
        print(f'waste --> {round(value(b_unit.waste.flow_vol[0.0]), 4) * 3600}\n')
        for py, ex in dict(zip(py_var, excel_var)).items():
            print(f'\n#### {py}')
            try:
                num = value(getattr(b_unit.costing, py))
                py_vals.append(num)
                print(f'Python --> {num}')
            except ZeroDivisionError:
                print(f'Python --> {0} - ERROR')
                py_vals.append(0)
            py_dict = dict(zip(py_var, py_vals))
            df = pd.read_csv("data/case_study_results.csv")
            df = df[df.case_study == train['case_study']]
            df = df[df.scenario == train['scenario']]
            df = df[df.unit_process == unit]
            num = df[df.Variable == ex].value.max()
            print(f'Excel --> {num}\n\n')
#             print(f'Flow In --> {num}\n\n')
            excel_vals.append(num)
            excel_dict = dict(zip(py_var, excel_vals))
        big_dict[unit]['python'] = py_dict
        big_dict[unit]['excel'] = excel_dict
else:
    pass 

df = pd.DataFrame.from_dict({(i,j): big_dict[i][j] 
                           for i in big_dict.keys() 
                           for j in big_dict[i].keys()},
                       orient='index')



____________well_field____________

flow in --> 1026.3600000000001
flow out --> 1026.3600000000001
waste --> 0.0


#### fixed_cap_inv
Python --> 2.7808393196787877
Excel --> 2.696919644



#### total_cap_investment
Python --> 2.924052544642245
Excel --> 2.835811006



#### cat_and_chem_cost
Python --> 0.0
Excel --> 0.0



#### electricity_cost
Python --> 0.0
Excel --> 0.122254216



#### total_fixed_op_cost
Python --> 0.03879270850951909
Excel --> 0.039162897


____________treated_storage____________

flow in --> 1104.12
flow out --> 1104.12
waste --> -0.0


#### fixed_cap_inv
Python --> 1.4874643471840963
Excel --> 9.180934336



#### total_cap_investment
Python --> 1.5640687610640773
Excel --> 9.653752455



#### cat_and_chem_cost
Python --> 0.0
Excel --> 0.0



#### electricity_cost
Python --> 0.0
Excel --> 0.0



#### total_fixed_op_cost
Python --> 0.020754223044643274
Excel --> 0.131609515


____________media_filtration____________

flow in --> 1104.12
flow out --> 1104.12
waste -

In [13]:
value(m.fs.treated_storage.flow_vol_in[0])

11.543350491578382

In [ ]:
# df_final = pd.DataFrame()
# for case_name in ["Ashkelon", "Carlsbad", "Tampa_Bay"]: #, "Santa_Barbra"]: #, "Ashkelon"]:

#     m = wt.watertap_setup(dynamic = False)

#     wt.case_study_trains.train = {"case_study": case_name,
#                                  "reference": "NAWI",
#                                  "scenario": "Baseline"}

#     # TODO LATER: how to make this sync with info in train input data. We might not need to do that.
#     #But, if the source water type is different to what is in the train (pfd dictionary), 
#     #then we should updat the node name. If more than two sources - what to do? Needs to be
#     #based on pfd node!?

#     wt.case_study_trains.source_water = {"case_study": case_name, 
#                                  "reference": "NAWI",
#                                  "scenario": "Baseline",
#                                  "water_type": "Seawater"}
    
#     m = wt.case_study_trains.get_case_study(m = m) # flow is set as case study flow unless defined.
    
      
#     wt.run_water_tap(m = m, solver_results = False, print_model_results = False)
#     print(value(m.fs.costing.LCOW))
#     df = get_results_table()
#     df["case_study"] = case_name
#     df_final = pd.concat([df_final,df])
    

In [5]:
def get_results_table():

    train = wt.case_study_trains.train

    py_var = [
        "fixed_cap_inv",
        "total_cap_investment",
        "cat_and_chem_cost",
        "electricity_cost",
        "total_fixed_op_cost"]

    excel_var = ['Fixed Capital Investment (FCI)',
                'Total Capital Investment (TCI)',
                'Catalysts and Chemicals',
                'Electricity',
                'Total Fixed Operating Costs']
    big_dict = {}
    # for letter in ['A', 'B', 'C']:
    for b_unit in m.fs.component_objects(Block, descend_into=True):
        unit = str(b_unit)[3:]
        if hasattr(b_unit, 'costing'):
            #print(f'____________{unit}____________')
            py_vals =  []
            excel_vals = []
            big_dict[unit] = {'python': {}, 'excel': {}}
            for py, ex in dict(zip(py_var, excel_var)).items():
                #print(f'\n#### {py}')
                try:
                    num = value(getattr(b_unit.costing, py))
                    py_vals.append(num)
                    #print(f'Python --> {num}')
                except ZeroDivisionError:
                    #print(f'Python --> {0} - ERROR')
                    py_vals.append(0)
                py_dict = dict(zip(py_var, py_vals))
                df = pd.read_csv("data/case_study_results.csv")
                df = df[df.case_study == train['case_study']]
                df = df[df.scenario == train['scenario']]
                df = df[df.unit_process == unit]
                num = df[df.Variable == ex].value.max()
                #print(f'Excel --> {num}\n\n')
                excel_vals.append(num)
                excel_dict = dict(zip(py_var, excel_vals))
            big_dict[unit]['python'] = py_dict
            big_dict[unit]['excel'] = excel_dict
    else:
        pass 



    df = pd.DataFrame.from_dict({(i,j): big_dict[i][j] 
                               for i in big_dict.keys() 
                               for j in big_dict[i].keys()},
                           orient='index')
    
    return df

In [6]:
get_results_table()

ERROR: evaluating object as numeric value:
    fs.surface_discharge.costing.electricity_cost
        (object: <class 'pyomo.core.base.expression.SimpleExpression'>)
    float division by zero


fixed_cap_inv  total_cap_investment  \
well_field                 python       2.780839              2.924053   
                           excel        2.696920              2.835811   
treated_storage            python       1.422587              1.495850   
                           excel        9.180934              9.653752   
media_filtration           python       3.228260              3.394516   
                           excel        2.424180              2.549025   
hydrochloric_acid_addition python       0.075211              0.079085   
                           excel        0.074794              0.078646   
anti_scalant_addition      python       0.049009              0.051533   
                           excel        0.121369              0.127619   
sodium_bisulfite_addition  python       0.075525              0.079414   
                           excel        0.096489              0.101459   
electrodialysis_reversal   python      35.783813             37.626679   
                           excel       38.458658             40.439279   
treated_storage_b          python       1.416026              1.488952   
                           excel             NaN                   NaN   
irwin_brine_management     python      36.532287             38.413700   
                           excel       42.345784             44.526591   
ro_deep                    python       0.600666              0.631600   
                           excel        8.443035              8.877851   
municipal_drinking         python       0.688765              0.724237   
                           excel        0.640832              0.673835   
deep_well_injection        python       1.350489              1.420039   
                           excel             NaN                   NaN   
surface_discharge          python       0.000000              0.000000   
                           excel             NaN                   NaN   

                                   cat_and_chem_cost  electricity_cost  \
well_field                 python           0.000000      0.000000e+00   
                           excel            0.000000      1.222542e-01   
treated_storage            python           0.000000      0.000000e+00   
                           excel            0.000000      0.000000e+00   
media_filtration           python           0.000000      0.000000e+00   
                           excel            0.000000      0.000000e+00   
hydrochloric_acid_addition python           0.022173      8.284925e-07   
                           excel            0.014850      8.200000e-07   
anti_scalant_addition      python           0.227603      4.142462e-07   
                           excel            0.466425      1.800000e-06   
sodium_bisulfite_addition  python           0.046955      8.340839e-07   
                           excel            0.048113      1.240000e-06   
electrodialysis_reversal   python           0.000000     -3.583309e-01   
                           excel            0.000000      3.607682e-01   
treated_storage_b          python           0.000000      0.000000e+00   
                           excel                 NaN               NaN   
irwin_brine_management     python           0.000000      1.234454e-03   
                           excel            0.000000      0.000000e+00   
ro_deep                    python           0.000000      1.413665e-02   
                           excel            0.000000      1.686516e-01   
municipal_drinking         python           0.000000      3.675403e-01   
                           excel            0.000000      3.379351e-01   
deep_well_injection        python           0.000000      3.727760e-03   
                           excel                 NaN               NaN   
surface_discharge          python           0.000000      0.000000e+00   
                           excel                 NaN               NaN   

                                   to

In [ ]:
df.loc['backwash_solids_handling']

In [ ]:
df = get_results_table()

In [ ]:
df.to_csv("tmp/test_compare.csv")

In [ ]:
m.fs.backwash_solids_handling.costing.electricity_cost()

In [ ]:
cat_chem_df = pd.read_csv('data/catalyst_chemicals.csv', index_col = "Material")

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("data/case_study_results.csv")

In [ ]:
wt.case_study_trains.train

In [ ]:
train = wt.case_study_trains.train

In [ ]:
df[df.case_study == train['case_study']]
df[df.scenario == train['scenario']]
df[df.unit_process == str(b_unit)[3:]]
print(df[df.Variable == "Fixed Capital Investment (FCI)"])

In [ ]:
print(b_unit)

In [ ]:
Excel_variable
Python_variable
Unit

Fixed Capital Investment (FCI)
fixed_cap_inv
$MM

Catalysts and Chemicals
cat_and_chem_cost
$MM/yr

Electricity
electricity_cost
$MM/yr

Other Operating
other_var_cost
$MM/yr

Total Fixed Operating 
total_fixed_op_cost
$MM/yr

Levelized Cost
LCOW
$/m3






In [ ]:
cost_range_list = []; #results will be inputted in this array
#up_name = "tri_media_filtration" # which unit process it applies to. TODO hould be user input.

#for value_change in pct_to_target1: # cycles through each value from MC range
for value_change in [0.4, 0.8]: #, 0.9]:

    # create and build model
    m = wt.watertap_setup(dynamic = False)
    m = wt.case_study_trains.get_case_study(name = 'carlsbad', flow = 4.5833, m = m)

    m.fs.tri_media_filtration.water_recovery.fix(value_change)

    # set variable to MC value
    wt.run_water_tap(m)
    results_table = get_results_table(m, unit_process_name)
    cost_range_list.append(results_table.total_up_cost.sum())


In [ ]:
cost_range_list

In [ ]:
#### DO NOT USE THE BELOW ####

In [ ]:
import time
from multiprocessing import Pool
import multiprocessing

mu = 0.6
sigma = .1
num_reps = 50

input_list = np.random.normal(mu,sigma, size = num_reps) #, sigma, num_reps).round(4)

count, bins, ignored = plt.hist(input_list, 25, density=True)
plt.plot(bins, 1/(sigma * np.sqrt(2 * np.pi)) * np.exp( - (bins - mu)**2 / (2 * sigma**2) ),
          linewidth=2, color='r')
plt.show()

### INPUT TO MODEL LIST: ### CAN BE AUTOMATED FOR USER TO LABEL THE VARIABLE. TOOD ###
no_of_proc = 4
list_final = []
for i in range(no_of_proc):
    part2 = len(input_list) / no_of_proc
    i2 = ((i+1)*part2)
    list1 = input_list[int(i*part2):int(i2)]
    list_final.append(list1)
    
    
def monte_run(list_final):
    print('goes in')

    up_name = "tri_media_filtration" # which unit process it applies to. TODO hould be user input.
    cost_range_list = []; #results will be inputted in this array

    #for value_change in pct_to_target1: # cycles through each value from MC range
    for value_change in list_final:

        # create and build model
        m = wt.watertap_setup(dynamic = False)
        m = wt.case_study_trains.get_case_study(name = 'carlsbad', flow = 4.5833, m = m)

        getattr(m.fs, up_name).water_recovery.fix(value_change)

        # set variable to MC value
        result = wt.run_water_tap(m)
        results_table = get_results_table(m, unit_process_names)
        cost_range_list.append(results_table.total_up_cost.sum())


    return cost_range_list

startTime = time.time()

pool=Pool()
dfs = pool.map(monte_run, list_final) #SomeClass().preprocess_data()

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

In [ ]:
####TO DO LOAD AND SAVE!!

In [ ]:
#### SAVE TRAIN ####
# path = 'trains/Tutorial1_treatment_train_example.csv'
# wt.save_train(T, path)

In [ ]:
# #### LOAD TRAIN ####
# path = 'trains/Tutorial1_treatment_train_example.csv'
# TT = wt.load_train(path)

In [ ]:
# wt.display.show_train(TT)